# Step 1: Introduction

---

For this project, I am evaluating the price of houses given the following features:

1. Year of sale of the house
2. The age of the house at the time of sale
3. Distance from city center
4. Number of stores in the locality
5. The latitude
6. The longitude

![Regression](images/regression.png)

Note: This notebook uses `python 3` and these packages: `tensorflow`, `pandas`, `matplotlib`, `scikit-learn`.

##Importing Libraries & Helper Functions



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from utils import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

print('Libraries imported.')

# Step 2: Importing the Data


The dataset is saved in a `data.csv` file. Used `pandas` to take a look at some of the rows.

In [ ]:
df = pd.read_csv('data.csv', names = column_names) 
df.head()

Check Missing Data



In [ ]:
df.isna().sum()

# Step 3: Data Normalization

## 
Normalizing the data before training a model makes it easier for optimization algorithms to find minimas. Normalization is changing the distribution of features so that the values for these features are in similar ranges

In [ ]:
df = df.iloc[:,1:]
df_norm = (df - df.mean()) / df.std()
df_norm.head()

## Convert Label Value

Convert the predicted values back to the original distribution to get predicted prices.

In [ ]:
y_mean = df['price'].mean()
y_std = df['price'].std()

def convert_label_value(pred):
    return int(pred * y_std + y_mean)

print(convert_label_value(0.350088))

# Step 4: Create Training and Test Sets

## Select Features

Remove the column __price__ from the list of features as it is the label and should not be used as a feature.

In [ ]:
X = df_norm.iloc[:, :6]
X.head()

## Select Labels

In [ ]:
Y = df_norm.iloc[:, -1]
Y.head()

## Feature and Label Values

Extracting just the numeric values for the features and labels as the TensorFlow model will expect just numeric values as input.

In [ ]:
X_arr = X.values
Y_arr = Y.values

print('X_arr shape: ', X_arr.shape)
print('Y_arr shape: ', Y_arr.shape)

## Train and Test Split

We will keep 5% of the data aside as a __test__ set for checking the performance of the model in trained and un-trained states.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size = 0.05, shuffle = True, random_state=0)

print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

# Step 5: Create the Model

##
get_model() is a function that returns an untrained model of a certain architecture.

In [ ]:
def get_model():
    
    model = Sequential([
        Dense(10, input_shape = (6,), activation = 'relu'),
        Dense(20, activation = 'relu'),
        Dense(5, activation = 'relu'),
        Dense(1)
    ])

    model.compile(
        loss='mse',
        optimizer='adadelta'
    )
    
    return model

model = get_model()
model.summary()

# Step 6: Model Training

##
Used an `EarlyStopping` callback from Keras to stop the model training if the validation loss stops decreasing for 5 epochs.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience = 5)

model = get_model()

preds_on_untrained = model.predict(X_test)

history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs = 1000,
    callbacks = [early_stopping]
)

## Plot Training and Validation Loss

The `plot_loss` helper function compares training and validation loss.

In [ ]:
plot_loss(history)

# Step 7: Predictions

## Plot Raw Predictions

The `compare_predictions` helper function can compare predictions from the model when it was untrained and when it was trained.

In [ ]:
preds_on_trained = model.predict(X_test)

compare_predictions(preds_on_untrained, preds_on_trained, y_test)

##Plot Price Predictions

The plot for price predictions and raw predictions will look the same with just one difference: The x and y axis scale is changed.

In [ ]:
price_on_untrained = [convert_label_value(y) for y in preds_on_untrained]
price_on_trained = [convert_label_value(y) for y in preds_on_trained]
price_y_test = [convert_label_value(y) for y in y_test]

compare_predictions(price_on_untrained, price_on_trained, price_y_test)